In [1]:
from utils import time_op
import impmeas as imp

solver = imp.formulas.GPMC()
imp.formulas.set_pmc_solver(solver)

In [2]:
# tests the model counting capabilities of SharpSAT-TD and Buddy

n,m,k = 8, 10, 7
cnf, formula = imp.random_k_cnf(n,m,k)
vars = [f"x{idx}" for idx in range(1, n+1)]

satcount_gpmc = time_op(solver.satcount)(cnf)
print(f"GPMC satcount = {satcount_gpmc}")

imp.formulas.set_buddy_context(vars)
f = time_op(imp.BuddyNode.parse)(formula)
satcount_bdd = time_op(f.expectation)() * 2**n
print(f"BDD satcount = {satcount_bdd}")

[00007.0608 ms / 0.0071 s / 0.0001 min] satcount
GPMC satcount = 248
[00078.7666 ms / 0.0788 s / 0.0013 min] parse
[00000.0081 ms / 0.0000 s / 0.0000 min] expectation
BDD satcount = 248.0


In [3]:
n,clauses_to_variables,k = 10, 4.5, 7
m = int(n*clauses_to_variables)
cnf, formula = imp.random_k_cnf(n,m,k)
vars = [f"x{idx+1}" for idx in range(n)]

In [4]:
import impmeas.bdds as bdd 
imp.formulas.set_buddy_context(vars)

x = "x1"
f = time_op(imp.BuddyNode.parse)(formula)
print(f"bdd size = {f.nodecount}")
blame = time_op(imp.blame)(f, x, rho=lambda x: 1/(x+1), cutoff=1e-4, alternative=True, debug=True)
infl = time_op(imp.influence)(f, x)
print(f"influence {infl:.5f}, {blame:.5f} <= blame")

[00557.6530 ms / 0.5577 s / 0.0093 min] parse
bdd size = 61
=== COMPUTING BLAME for x1 in <impmeas.formulas.buddy.BuddyNode object at 0x7f3c51f199c0> ===
k=0 d result=0.0781 max increase possible=0.4609 
k=1 d result=0.1797 max increase possible=0.1875 
k=2 d result=0.1510 max increase possible=0.0273 
k=3 d result=0.0269 max increase possible=0.0004 
k=4 d result=0.0004 max increase possible=0.0000 
stopped earlier because cannot improve above cutoff=0.0001.
current value: 0.4361, can only be increased by 0.0000.
=== DONE ===
[00025.2078 ms / 0.0252 s / 0.0004 min] blame
[00000.0415 ms / 0.0000 s / 0.0000 min] influence
influence 0.07812, 0.43610 <= blame


In [5]:
x = "x1"
f = time_op(imp.Formula.parse)(formula)
blame = time_op(imp.blame)(f, x, rho=lambda x: 1/(x+1), cutoff=1e-4, alternative=True, debug=True)
infl = (time_op)(imp.influence)(f, x)
print(f"influence {infl:.5f}, {blame:.5f} <= blame")

[00583.3607 ms / 0.5834 s / 0.0097 min] parse
=== COMPUTING BLAME for x1 in Formula with size 1755 ===
k=0 size of cnf: 1180 d result=0.0781 max increase possible=0.4609 
k=1 size of cnf: 1211 d result=0.1797 max increase possible=0.1875 
k=2 size of cnf: 1238 d result=0.1510 max increase possible=0.0273 
k=3 size of cnf: 1261 d result=0.0269 max increase possible=0.0004 
k=4 size of cnf: 1280 d result=0.0004 max increase possible=0.0000 
stopped earlier because cannot improve above cutoff=0.0001.
current value: 0.4361, can only be increased by 0.0000.
=== DONE ===
[00318.8832 ms / 0.3189 s / 0.0053 min] blame
[00150.4211 ms / 0.1504 s / 0.0025 min] influence
influence 0.07812, 0.43610 <= blame


In [6]:

n,clauses_to_variables,k = 10, 4.5, 7
m = int(n*clauses_to_variables)
cnf, formula = imp.random_k_cnf(n,m,k)
vars = [f"x{idx+1}" for idx in range(n)]
x = "x1"

f = time_op(imp.Table.parse)(formula)
blame = time_op(imp.blame)(f, x, rho=lambda x: 1/(x+1), cutoff=1e-4, debug=False)
print(f"{blame:.5f} <= blame")
infl = time_op(imp.influence)(f, x)
print(f"influence = {infl:.5f}")
hammer = time_op(lambda f,x: imp.banzhaf(imp.hkr_cgm(f),x))(f, x)
print(f"hammer = {hammer:.5f}")
omega = time_op(imp.dominating_cgm)(f)
upsilon = time_op(imp.rectifying_cgm)(f)

print("="*100)
imp.formulas.set_buddy_context(vars)

f = time_op(imp.BuddyNode.parse)(formula)
print(f"bdd size = {f.nodecount}")
blame = time_op(imp.blame)(f, x, rho=lambda x: 1/(x+1), cutoff=1e-4, debug=True)
infl = time_op(imp.influence)(f, x)
print(f"influence {infl:.5f}, {blame:.5f} <= blame")


[00893.3971 ms / 0.8934 s / 0.0149 min] parse
[01373.5821 ms / 1.3736 s / 0.0229 min] blame
0.63525 <= blame
[00013.1588 ms / 0.0132 s / 0.0002 min] influence
influence = 0.34375
[06571.2042 ms / 6.5712 s / 0.1095 min] <lambda>
hammer = 0.07769
[00746.5584 ms / 0.7466 s / 0.0124 min] dominating_cgm
[00687.0615 ms / 0.6871 s / 0.0115 min] rectifying_cgm
[00619.1201 ms / 0.6191 s / 0.0103 min] parse
bdd size = 94
=== COMPUTING BLAME for x1 in <impmeas.formulas.buddy.BuddyNode object at 0x7f3c51f199c0> ===
k=0 d result=0.3438 max increase possible=0.3281 
k=1 d result=0.2227 max increase possible=0.0703 
k=2 d result=0.0645 max increase possible=0.0044 
k=3 d result=0.0044 max increase possible=0.0000 
stopped earlier because cannot improve above cutoff=0.0001.
current value: 0.6353, can only be increased by 0.0000.
=== DONE ===
[00163.1837 ms / 0.1632 s / 0.0027 min] blame
[00000.0658 ms / 0.0001 s / 0.0000 min] influence
influence 0.34375, 0.63525 <= blame
